In [1]:
import pandas as pd
import csv
import sys
import requests
from lxml import html as htmlRenderer
csv.field_size_limit(sys.maxsize)

131072

In [19]:
csv_file_in = "/home/cflores/cflores_workspace/comments-retriever/results/20minutos/20minutos-01012019-15092019-contents.csv"
csv_file_out = "/home/cflores/cflores_workspace/comments-retriever/results/20minutos/20minutos-01012019-15092019-contents-fixed.csv"

In [20]:
def is_valid_csv(csv_file):
    invalid_rows = []
    no_content = []
    no_title = []
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        counter = 0
        for r in reader:
            if not r[0]:
                print(" row -{}- row has not url.".format(counter))
                invalid_rows.append(counter)
            if not r[1]: 
                print(" row -{}- row has not content.".format(counter))
                invalid_rows.append(counter)
                no_content.append(r[0])
            if not r[2]:
                print(" row -{}- row has not title.".format(counter))
                invalid_rows.append(counter)
                no_title.append(r[0])
            counter = counter + 1
        
    return len(list(dict.fromkeys(invalid_rows))) == 0, invalid_rows, no_content, no_title

In [21]:
def get_title(url):
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    queryXpath = "//div[@id='main']//h1[@class='article-title']"
    el = renderedPage.xpath(queryXpath)
    title = url
    if len(el) == 0:
        print(" -> title not found in: {}".format(url))
    else:
        title = el[0].text
    return title

In [25]:
def extract_content(url=None):
    queries_xpath = ["//div[@class='gtm-article-text']//p", 
                     "//div[@class='gtm-article-text']//span",
                     "//div[@class='gtm-article-text']/div[@class='gmail_default']"]
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    
    contentStr = ""
    for q in queries_xpath:
        commentsElList = renderedPage.xpath(q)
        if len(commentsElList) > 0:
            contentArr = []
            for p in commentsElList:
                contentArr.append(p.text_content())
            contentStr = "".join([parrafo for parrafo in contentArr])
            break
    if not contentStr:
        print("\t -> url has not content found {}".format(url))
    return contentStr

In [26]:
keys = []
no_content = []
no_title = []
with open(csv_file_in, 'r') as f:
    with open(csv_file_out, 'w') as fout:
        writer = csv.writer(fout)
        reader = csv.reader(f)
        for r in reader:
            if not r[1]:
                # no content
                print(" => row with url {} has any content".format(r[0]))
                content = extract_content(r[0])
                if content:
                    print(" \t ===>>> content found ")
                r[1] = content
            if not r[2] or r[2] == r[0]:
                # no title
                print(" => row with url {} has any title".format(r[0]))
                r[2] = get_title(r[0])
            
            writer.writerow(r)    

 => row with url https://www.20minutos.es/noticia/3541255/0/totalan/ has any content
	 -> url has not content found https://www.20minutos.es/noticia/3541255/0/totalan/
 => row with url https://www.20minutos.es/noticia/3556752/0/illuminati-dominaran-mundo/ has any content
	 -> url has not content found https://www.20minutos.es/noticia/3556752/0/illuminati-dominaran-mundo/
 => row with url https://www.20minutos.es/noticia/3556752/0/illuminati-dominaran-mundo/ has any title
 -> title not found in: https://www.20minutos.es/noticia/3556752/0/illuminati-dominaran-mundo/
 => row with url https://www.20minutos.es/noticia/3568804/0/sanidad-alerta-escasez-konakion-medicamento-considerado-esencial-oms/ has any content
 	 ===>>> content found 
 => row with url https://www.20minutos.es/noticia/3597306/0/directo-juicio-proces-26-marzo-2019-guardia-civil-1o/ has any content
	 -> url has not content found https://www.20minutos.es/noticia/3597306/0/directo-juicio-proces-26-marzo-2019-guardia-civil-1o/


In [11]:
valid, invalids, no_content, no_title = is_valid_csv(csv_file_in)

 row -1793- row has not content.
 row -3774- row has not content.
 row -5246- row has not content.
 row -8845- row has not content.
 row -12351- row has not content.


In [27]:
valid, invalids, no_content, no_title = is_valid_csv(csv_file_out)

 row -1793- row has not content.
 row -3774- row has not content.
 row -8845- row has not content.


In [28]:
no_content

['https://www.20minutos.es/noticia/3541255/0/totalan/',
 'https://www.20minutos.es/noticia/3556752/0/illuminati-dominaran-mundo/',
 'https://www.20minutos.es/noticia/3597306/0/directo-juicio-proces-26-marzo-2019-guardia-civil-1o/']

In [29]:
wrong_url = "https://www.20minutos.es/noticia/3544280/0/juan-guaido-presidente-interino-venezuela/"
with open(csv_file_in, 'r') as f:
    reader = csv.reader(f)
    counter = 0
    for r in reader:
        if r[0] == wrong_url:
            print(r[1])
            break

Los dos presidentes de Venezuela, Nicolás Maduro y Juan Guaidó, se han contraprogramado este viernes en Caracas. Mientras el aún mandatario del país ha convocado una rueda de prensa en el Palacio de Miraflores, el líder de la Asamblea Nacional, que el miércoles declaró que asumía las competencias del Ejecutivo, dictó un discurso en Chacao, cerca de la capital.Maduro ha respondido a la exigencia formulada por el ministro de Asuntos Exteriores español, Josep Borrell, de convocar elecciones cuanto antes en el país amenazando de lo contrario con reconocer a Juan Guaidó como mandatario venezolano que lo que debe hacer España es convocar elecciones ya que Pedro Sánchez no fue elegido en las urnas."Si ellos quieren elecciones que las hagan en España" puesto que el presiente del Gobierno "no fue electo en ningún voto popular", ha replicado Maduro en rueda de prensa en el Palacio de Miraflores. "No pueden dar lecciones ni pueden poner ningún ultimátum", ha sostenido.En Chacao, al este de Caraca